In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
#collect the images and put them in dataframe
with_mask_images = []
without_mask_images = []
i = 0
for file_name in os.listdir("../input/face-mask-dataset/data/with_mask"):
    image =tf.keras.preprocessing.image.load_img(os.path.join("../input/face-mask-dataset/data/with_mask",file_name),target_size=(50,50))
    image = tf.keras.preprocessing.image.img_to_array(image, dtype="float32")
    image=tf.keras.applications.mobilenet_v2.preprocess_input(image)
    with_mask_images.append(image)
    i = i + 1
    if(i > 3500):
        break
j = 0   
for file_name in os.listdir("../input/face-mask-dataset/data/without_mask"):
    image =tf.keras.preprocessing.image.load_img(os.path.join("../input/face-mask-dataset/data/without_mask",file_name),target_size=(50,50))
    image = tf.keras.preprocessing.image.img_to_array(image, dtype="float32")
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    without_mask_images.append(image)
    j = j + 1
    if(j > 3500):
        break
    
df = pd.DataFrame({"images":with_mask_images, "with_mask":[1 for element in with_mask_images]})
without_mask_df = pd.DataFrame({"images":without_mask_images, "with_mask":[0 for element in without_mask_images]})

In [3]:
df = df.append(without_mask_df, ignore_index = True)
df.sample(frac = 1 ,ignore_index = True, random_state = 1)  #shuffle the dataframe


In [4]:
#divide the data to train, cross validation and test sets
x_train, x_test, y_train, y_test  = train_test_split(df["images"],df["with_mask"],test_size=0.4,shuffle = True, random_state=42)
x_cross_validation, x_test, y_cross_validation, y_test  = train_test_split(x_test,y_test,test_size=0.5,shuffle = True, random_state=42)

In [10]:
x_train = np.array([np.array(val) for val in x_train])
x_test = np.array([np.array(val) for val in x_test])
x_cross_validation = np.array([np.array(val) for val in x_cross_validation])

In [12]:
#build the model
model = tf.keras.models.Sequential();

model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu',input_shape=(50,50,3)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Flatten());
model.add(tf.keras.layers.Dense(50, activation= "relu"));
model.add(tf.keras.layers.Dense(50, activation= "relu"));
model.add(tf.keras.layers.Dense(2, activation= "softmax"));
model.compile(optimizer = "adam",
              loss = "sparse_categorical_crossentropy",
              metrics = ['accuracy'])
history = model.fit(x_train, y_train,batch_size=30, epochs = 5, validation_data = (x_cross_validation, y_cross_validation),workers=4)

In [13]:
#visualize the loss with the train and crossvalidation
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label = 'Cross Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 1])
plt.legend(loc='upper right')


In [14]:
val_loss, val_accuracy = model.evaluate(x_test, y_test)
print(val_loss,val_accuracy)

In [15]:
#extract the output from the probability distrbution
predictions = np.argmax(model.predict(x_test),axis = 1)
#show the precision, recall and f1_score
print(classification_report(y_test, predictions))

In [16]:
predictions_classes = ["without_mask", "with_mask"]
plt.imshow(x_test[0])
plt.show()
print("the predicted class is "+ predictions_classes[predictions[0]])
